In [4]:
#Below is a Python script, that will pull data directly from the spreadsheet in question,  manipulate and merge the data across the 3 Raw Data tabs and in the Mapping Table tab, and redirect the results back to a new tab, called Pivot_Data


In [ ]:
#importing required modules

import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize


#Assigning variable to the link of the sheet in question

scope = ['https://docs.google.com/spreadsheets/d/1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8/edit?usp=sharing']


#https://docs.google.com/spreadsheets/d/1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8/edit?usp=sharing

credentials = ServiceAccountCredentials.from_json_keyfile_name('./cool-benefit-290520-30a2c738b36d (1).json', scope)
gc = gspread.authorize(credentials)
gc = gspread.service_account(filename='./cool-benefit-290520-30a2c738b36d (1).json')
sh = gc.open_by_key('1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8')



#This will allow us to target data specific to each tab
Pivot_Data_worksheet = sh.get_worksheet(2)
Google_Ads_worksheet = sh.get_worksheet(3)
Bing_Ads_worksheet = sh.get_worksheet(4)
Media_Plan_worksheet = sh.get_worksheet(5) 
Mapping_Table_worksheet = sh.get_worksheet(6)

In [11]:
#turning each dataset into a dataframe

Google_list = Google_Ads_worksheet.get_all_values()
Google_frame = pd.DataFrame(Google_list,  columns = Google_list[0])
Google_Ads = Google_frame.drop(Google_frame.index[0])

Bing_list = Bing_Ads_worksheet.get_all_values()
Bing_frame = pd.DataFrame(Bing_list,  columns = Bing_list[0])
Bing_Ads = Bing_frame.drop(Bing_frame.index[0])

Media_list = Media_Plan_worksheet.get_all_values()
Media_frame = pd.DataFrame(Media_list,  columns = Media_list[0])
Media_table = Media_frame.drop(Media_frame.index[0])

Mapping_list = Mapping_Table_worksheet.get_all_values()
Mapping_frame = pd.DataFrame(Mapping_list,  columns = Mapping_list[0])
Mapping_table = Mapping_frame.drop(Mapping_frame.index[0])

In [12]:
Google_Ads.shape
Bing_Ads.shape

(496, 13)

In [14]:
#Merging data from distinct tables, based on common values

Mapped_Media = Mapping_table.merge(Media_table, left_on = ['Corresponding Media Plan Line Id','Platform'], right_on = ['Plan Line Id', 'Platform'], how = 'inner')
Mapped_Media_Google = Mapped_Media.loc[Mapped_Media['Platform'] == 'Google Ads']
Mapped_Media_Bing = Mapped_Media.loc[Mapped_Media['Platform'] == 'Bing Ads']

In [15]:
#Mapped_Media_Bing
#Mapped_Media_Google

#Mapped_Media_Total = pd.concat([Mapped_Media_Bing,Mapped_Media_Google],ignore_index=True)
#Mapped_Media_Total

In [16]:
import numpy as np

Bing_Ads.columns

Bing_spend = Bing_Ads['Spend'].str.replace('$', '').fillna(0).to_list()
Bing_spend = pd.to_numeric(Bing_spend)



Bing_Ads['Amount Spent'] = Bing_spend
Bing_Ads['Cost_per_Acquisition'] = Bing_Ads['Amount Spent']/pd.to_numeric(Bing_Ads['Conversions'])
Bing_Ads['Cost_per_Acquisition'] = Bing_Ads['Cost_per_Acquisition'].round(2)
Bing_Ads['Cost_per_Acquisition'] = Bing_Ads['Cost_per_Acquisition'].fillna(0)
Bing_Ads = Bing_Ads.replace([np.inf, -np.inf], np.nan)
Bing_Ads = Bing_Ads.fillna(0)

Bing_cols = Bing_Ads[['Campaign Name', 'Amount Spent', 'Conversions', 'Search Imp. Share', 'Cost_per_Acquisition']]
Bing_cols.shape

#df.replace([np.inf, -np.inf], np.nan)
#Bing_Ads['Cost_per_Acquisition'] = Bing_Ads['Cost_per_Acquisition']
#Bing_Ads['Cost_per_Acxxx'] = Bing_Ads['Cost_per_Acxxx'].round(2)                                                    

#Bing_cols = Bing_Ads[['Campaign Name', 'Amount Spent', 'Conversions', 'Search Imp. Share', 'Cost_per_Acquisition']]
#Bing_cols['Cost_per_Acquisition'].replace([np.inf, -np.inf], np.nan)


(496, 5)

In [17]:
Google_spend = Google_Ads['Spend'].str.replace('$', '').fillna(0).to_list()

#Two elements were producing errors due to the comma within the numerical expression
Google_spend[357] = Google_spend[357].replace(',', '')
Google_spend[357]

Google_spend[421] = Google_spend[421].replace(',', '')
Google_spend[421]


Google_spend = pd.to_numeric(Google_spend)

Google_Ads['Amount Spent'] = Google_spend
Google_Ads['Cost_per_Acquisition'] = Google_Ads['Amount Spent']/pd.to_numeric(Google_Ads['Conversions'])
Google_Ads['Cost_per_Acquisition'] = Google_Ads['Cost_per_Acquisition'].round(2)
Google_Ads['Cost_per_Acquisition'] = Google_Ads['Cost_per_Acquisition'].fillna(0)
Google_Ads = Google_Ads.replace([np.inf, -np.inf], np.nan)
Google_Ads = Google_Ads.fillna(0)

Google_cols = Google_Ads[['Campaign Name', 'Amount Spent', 'Conversions', 'Search Imp. Share', 'Cost_per_Acquisition']]
Google_cols.shape


(992, 5)

In [18]:
Mapped_Media_Google
Mapped_Media_Bing
Google_cols
Bing_cols

,Campaign Name,Amount Spent,Conversions,Search Imp. Share,Cost_per_Acquisition
1,Product 1 | EMEA | GB | en | BKWS | BMM,53.71,1,77.84%,53.71
2,Product 1 | EMEA | GB | en | BKWS | BMM,2.35,1,100.00%,2.35
3,Product 1 | EMEA | GB | en | SKWS | BMM,52.09,4,97.45%,13.02
4,Product 1 | EMEA | GB | en | SKWS | BMM,35.35,7,71.09%,5.05
5,Product 2 | EMEA | GB | en | BKWS | BMM,15.85,1,100.00%,15.85
...,...,...,...,...,...
492,Product 1 | EMEA | GB | en | SKWS | Exact,0.00,0,-,0.00
493,Product 2 | EMEA | GB | en | BKWS | Exact,1.49,0,85.25%,0.00
494,Product 2 | EMEA | GB | en | BKWS | Exact,0.00,0,93.75%,0.00
495,Product 2 | EMEA | GB | en | SKWS | Exact,0.00,0,-,0.00


In [19]:
Google_Merged = Google_cols.merge(Mapped_Media_Google, left_on = ['Campaign Name'], right_on = ['Campaign'], how = 'inner')    
Bing_Merged = Bing_cols.merge(Mapped_Media_Bing, left_on = ['Campaign Name'], right_on = ['Campaign'], how = 'inner')    

#Bing_Merged.columns
#Mapping_table.merge(Media_table, left_on = ['Corresponding Media Plan Line Id','Platform'], right_on = ['Plan Line Id', 'Platform'], how = 'inner')

In [20]:
Google_Merged_Final = Google_Merged[['Platform', 'Campaign Name', 'Plan Line', 'Amount Spent', 'Conversions', 'Search Imp. Share', 'Cost_per_Acquisition']] 

In [21]:
Bing_Merged_Final = Bing_Merged[['Platform', 'Campaign Name', 'Plan Line', 'Amount Spent', 'Conversions', 'Search Imp. Share', 'Cost_per_Acquisition']] 

In [22]:
#Bing_Merged_Final

In [23]:
Ads_total = pd.concat([Google_Merged_Final,Bing_Merged_Final],ignore_index=True)

In [24]:
Ads_total
#Bing_Merged_Final.shape
#Google_Merged_Final.shape

,Platform,Campaign Name,Plan Line,Amount Spent,Conversions,Search Imp. Share,Cost_per_Acquisition
0,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,179.04,2,77.84%,89.52
1,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,7.84,1,100.00%,7.84
2,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,146.73,3,76.57%,48.91
3,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,4.87,1,97.14%,4.87
4,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,96.62,1,71.38%,96.62
...,...,...,...,...,...,...,...
1483,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,0.00,0,-,0.00
1484,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,0.00,0,-,0.00
1485,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,0.00,0,-,0.00
1486,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,0.00,0,-,0.00


In [25]:
Ads_total['Search Imp. Share'] = Ads_total['Search Imp. Share'].str.replace('%', '').fillna(0).to_list()
Ads_total['Search Imp. Share'] = Ads_total['Search Imp. Share'].str.replace('-', '')
Ads_total['Search Imp. Share'] = Ads_total['Search Imp. Share'].str.replace('<', '')

Ads_total['Search Imp. Share'] = pd.to_numeric(Ads_total['Search Imp. Share'])
#Google_spend = Google_Ads['Spend'].str.replace('$', '').fillna(0).to_list()

In [26]:
Ads_total
Final_Ads_total = Ads_total[Ads_total['Amount Spent']>0]
Final_Ads_total

,Platform,Campaign Name,Plan Line,Amount Spent,Conversions,Search Imp. Share,Cost_per_Acquisition
0,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,179.04,2,77.84,89.52
1,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,7.84,1,100.00,7.84
2,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,146.73,3,76.57,48.91
3,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,4.87,1,97.14,4.87
4,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,96.62,1,71.38,96.62
...,...,...,...,...,...,...,...
1469,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,34.01,0,39.34,0.00
1471,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,60.72,1,38.57,60.72
1473,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,75.63,0,44.71,0.00
1475,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,80.05,0,10.00,0.00


In [27]:
Final_Ads_total.columns

Index(['Platform', 'Campaign Name', 'Plan Line', 'Amount Spent', 'Conversions',
       'Search Imp. Share', 'Cost_per_Acquisition'],
      dtype='object')

In [28]:
#pivot_table = Final_Ads_total.pivot_table(values=['Search Imp. Share', 'Cost_per_Acquisition'], index = ['Platform'], aggfunc=np.mean)

#pivot_table

In [29]:
pivot_table = Final_Ads_total.pivot_table(values=['Search Imp. Share','Cost_per_Acquisition'], index = ['Plan Line'], aggfunc=np.mean)

In [30]:
pivot_table

,Cost_per_Acquisition,Search Imp. Share
Plan Line,,
Brand Keywords,22.171949,87.480847
Generic Keywords,27.709057,62.955906


In [31]:
Final_Ads_total

,Platform,Campaign Name,Plan Line,Amount Spent,Conversions,Search Imp. Share,Cost_per_Acquisition
0,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,179.04,2,77.84,89.52
1,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,7.84,1,100.00,7.84
2,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,146.73,3,76.57,48.91
3,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,4.87,1,97.14,4.87
4,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,96.62,1,71.38,96.62
...,...,...,...,...,...,...,...
1469,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,34.01,0,39.34,0.00
1471,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,60.72,1,38.57,60.72
1473,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,75.63,0,44.71,0.00
1475,Bing Ads,Product 2 | EMEA | GB | en | SKWS | Exact,Generic Keywords,80.05,0,10.00,0.00


In [32]:
#Final_Ads_total.


In [33]:
#I will now create a new column, specifying the country specifically. This will allow me to group the data across this variable in the future 

GB_Data = Final_Ads_total[Final_Ads_total["Campaign Name"].str.contains("GB", na=False)]  #found the solution for this here https://stackoverflow.com/questions/11350770/select-by-partial-string-from-a-pandas-dataframe
FR_Data = Final_Ads_total[Final_Ads_total["Campaign Name"].str.contains("FR", na=False)]  #found the solution for this here https://stackoverflow.com/questions/11350770/select-by-partial-string-from-a-pandas-dataframe


GB = []
for i in GB_Data['Campaign Name']:
    GB.append('GB')

FR = []
for i in FR_Data['Campaign Name']:
    FR.append('FR')
    
GB_Data['Country'] = GB
FR_Data['Country'] = FR


Final_Data_Set = pd.concat([GB_Data, FR_Data], ignore_index=True)
Final_Data_Set

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,Platform,Campaign Name,Plan Line,Amount Spent,Conversions,Search Imp. Share,Cost_per_Acquisition,Country
0,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,179.04,2,77.84,89.52,GB
1,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,7.84,1,100.00,7.84,GB
2,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,146.73,3,76.57,48.91,GB
3,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,4.87,1,97.14,4.87,GB
4,Google Ads,Product 1 | EMEA | GB | en | BKWS | BMM,Brand Keywords,96.62,1,71.38,96.62,GB
...,...,...,...,...,...,...,...,...
1106,Google Ads,Product 2 | EMEA | FR | fr | SKWS | Exact,Generic Keywords,3.13,0,18.49,0.00,FR
1107,Google Ads,Product 2 | EMEA | FR | fr | SKWS | Exact,Generic Keywords,1.00,0,67.89,0.00,FR
1108,Google Ads,Product 2 | EMEA | FR | fr | SKWS | Exact,Generic Keywords,2.96,0,18.32,0.00,FR
1109,Google Ads,Product 2 | EMEA | FR | fr | SKWS | Exact,Generic Keywords,0.95,0,57.76,0.00,FR


In [110]:
type(Final_Data_Set)

pandas.core.frame.DataFrame

In [111]:
#And now to send the data over to spreadsheets for additional analysis

Pivot_Data_worksheet.update([Final_Data_Set.columns.values.tolist()] + Final_Data_Set.values.tolist())

{'spreadsheetId': '1EecGKdMSqWXzTxfjEfYPCoK0ph-yAOSkWyoj_5z9an8',
 'updatedRange': 'Answers!A1:H1112',
 'updatedRows': 1112,
 'updatedColumns': 8,
 'updatedCells': 8896}

In [ ]:
#Below is an additional piece of analysis, which produces a pivot table from the tables above

In [105]:
pivot_table = Final_Data_Set.pivot_table(values=['Search Imp. Share','Cost_per_Acquisition'], index = ['Country'], columns = ['Plan Line'],margins = True, margins_name='Total', aggfunc=np.mean).round(2)

In [106]:
type(pivot_table)

pandas.core.frame.DataFrame

In [107]:
pivot_table

Cost_per_Acquisition                         Search Imp. Share  \
Plan Line       Brand Keywords Generic Keywords  Total    Brand Keywords   
Country                                                                    
FR                       23.58            17.14  21.12             77.92   
GB                       21.44            33.97  25.85             92.48   
Total                    22.17            27.71  24.18             87.48   

                                   
Plan Line Generic Keywords  Total  
Country                            
FR                   51.23  67.73  
GB                   69.91  84.52  
Total                62.96  78.58

In [ ]:
###############################################################################################################################